In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tweet-preprocessor
import preprocessor as p

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer,WordNetLemmatizer
from wordcloud import WordCloud
import re
import os
import gensim
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.utils import resample
from sklearn.metrics import accuracy_score,f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Load Dataset

In [ ]:
## before this, remember to find and replace “ with "  (take care of utf-8 encoding issue)
## before this, we can also utilize Openrefine to re-export the files to take care of utf-8 encoding issue
df = pd.read_csv("/content/drive/My Drive/01-Research/Geo-Isa/01-CleanedTwitterData/NewHampshire-counties.csv")
df.head()

,Date,Headline,URL,Opening Text,Hit Sentence,Source,Influencer,Country,Subregion,Language,...,Likes,Replies,Retweets,Comments,Shares,Reactions,Threads,Is Verified,Parent URL,Document Tags
0,16-Oct-2023 11:11PM,NaN,http://twitter.com/TomMcGuinness/statuses/1714...,NaN,"RT @cannafeminist: Grafton, you couldn’t find ...",Twitter,@TomMcGuinness,United States,NaN,English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,https://twitter.com/cannafeminist/statuses/171...,NaN
1,16-Oct-2023 03:14PM,NaN,http://twitter.com/AndiJohnson1/statuses/17139...,NaN,"RT @cannafeminist: Grafton, you couldn’t find ...",Twitter,@AndiJohnson1,United States,NaN,English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,https://twitter.com/cannafeminist/statuses/171...,NaN
2,16-Oct-2023 03:14PM,NaN,http://twitter.com/ChairmanBuckley/statuses/17...,NaN,"RT @cannafeminist: Grafton, you couldn’t find ...",Twitter,@ChairmanBuckley,United States,NaN,English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,https://twitter.com/cannafeminist/statuses/171...,NaN
3,16-Oct-2023 08:36AM,NaN,http://twitter.com/DavidMarkDC/statuses/171389...,NaN,Rep. Marjorie Taylor Greene (R-Ga.) to headlin...,Twitter,@DavidMarkDC,United States,NaN,English,...,0.0,4.0,0.0,NaN,0.0,0.0,4.0,False,NaN,NaN
4,15-Oct-2023 06:22PM,NaN,http://twitter.com/TheBethVarney/statuses/1713...,NaN,RT @KellyAyotte: Sullivan County is ready to k...,Twitter,@TheBethVarney,United States,NaN,English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,https://twitter.com/KellyAyotte/statuses/17136...,NaN


#   Preprocessing Tweet Text

1. Removing Twitter Handles (@user)
2. Removing urls from text
3. Removing Punctuations, Numbers, and Special Characters
5. Convert the word to lowercase
6. Remove Stopwords
7. Removing RT and QTs

In [ ]:
text=df['Hit Sentence'].values.tolist()
print(len(text))

5067


In [ ]:
def preprocess_tweet(row):
    text = row['Hit Sentence']
    text = p.clean(text)
    return text

In [ ]:
df = df[df["Hit Sentence"].str.contains("RT")==False] ## remove RTs
df = df[df["Hit Sentence"].str.contains("QT")==False] ## remove Quote tweets
df

,Date,Headline,URL,Opening Text,Hit Sentence,Source,Influencer,Country,Subregion,Language,...,Likes,Replies,Retweets,Comments,Shares,Reactions,Threads,Is Verified,Parent URL,Document Tags
3,16-Oct-2023 08:36AM,NaN,http://twitter.com/DavidMarkDC/statuses/171389...,NaN,Rep. Marjorie Taylor Greene (R-Ga.) to headlin...,Twitter,@DavidMarkDC,United States,NaN,English,...,0.0,4.0,0.0,NaN,0.0,0.0,4.0,False,NaN,NaN
6,15-Oct-2023 05:55PM,NaN,http://twitter.com/KellyAyotte/statuses/171367...,NaN,Sullivan County is ready to keep New Hampshire...,Twitter,@KellyAyotte,Unknown,NaN,English,...,31.0,3.0,2.0,NaN,2.0,31.0,3.0,True,NaN,NaN
13,14-Oct-2023 06:20AM,NaN,http://twitter.com/HamptonsDrone/statuses/1713...,NaN,New Hampshire Foliage & Cog Rail -Drone footag...,Twitter,@HamptonsDrone,United States,NaN,English,...,49.0,2.0,6.0,NaN,6.0,49.0,2.0,True,NaN,NaN
16,11-Oct-2023 07:02PM,NaN,http://twitter.com/leand13000/statuses/1712242...,NaN,"@VincentCrypt46 I'm in Rockingham County, New ...",Twitter,@leand13000,United States,NaN,English,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,False,http://twitter.com/VincentCrypt46/statuses/171...,NaN
21,11-Oct-2023 10:34AM,NaN,http://twitter.com/TownsUsa/statuses/171211443...,NaN,"Tilton Northfield, New Hampshire Belknap Count...",Twitter,@TownsUsa,United States,NaN,English,...,12.0,0.0,0.0,NaN,0.0,12.0,0.0,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5058,19-Oct-2022 10:19AM,NaN,http://twitter.com/nsarwark/statuses/158273833...,NaN,Hillsborough County (NH) had more Superior Cou...,Twitter,@nsarwark,United States,NaN,English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN
5060,18-Oct-2022 09:45PM,NaN,http://twitter.com/UnionLeader/statuses/158254...,NaN,A 29-year-old former Manchester resident will ...,Twitter,@UnionLeader,United States,NaN,English,...,7.0,2.0,NaN,NaN,0.0,7.0,2.0,False,NaN,NaN
5061,18-Oct-2022 02:19PM,NaN,http://twitter.com/letshearthetru/statuses/158...,NaN,@KrissOnAMission @miissdirection @CEARJustice ...,Twitter,@letshearthetru,Unknown,NaN,English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,http://twitter.com/letshearthetru/statuses/158...,NaN
5062,18-Oct-2022 08:46AM,NaN,https://twitter.com/ham_photos_bot/statuses/15...,NaN,"Unidentified Artist, Charity, Public: United S...",Twitter,@ham_photos_bot,United States,NaN,English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN


In [ ]:
df['Hit Sentence'] = df.apply(preprocess_tweet, axis=1)

In [ ]:
## Delete duplicates
df = df.drop_duplicates(subset='Hit Sentence', keep="first")
df

,Date,Headline,URL,Opening Text,Hit Sentence,Source,Influencer,Country,Subregion,Language,...,Likes,Replies,Retweets,Comments,Shares,Reactions,Threads,Is Verified,Parent URL,Document Tags
3,16-Oct-2023 08:36AM,NaN,http://twitter.com/DavidMarkDC/statuses/171389...,NaN,Rep. Marjorie Taylor Greene (R-Ga.) to headlin...,Twitter,@DavidMarkDC,United States,NaN,English,...,0.0,4.0,0.0,NaN,0.0,0.0,4.0,False,NaN,NaN
6,15-Oct-2023 05:55PM,NaN,http://twitter.com/KellyAyotte/statuses/171367...,NaN,Sullivan County is ready to keep New Hampshire...,Twitter,@KellyAyotte,Unknown,NaN,English,...,31.0,3.0,2.0,NaN,2.0,31.0,3.0,True,NaN,NaN
13,14-Oct-2023 06:20AM,NaN,http://twitter.com/HamptonsDrone/statuses/1713...,NaN,New Hampshire Foliage & Cog Rail -Drone footag...,Twitter,@HamptonsDrone,United States,NaN,English,...,49.0,2.0,6.0,NaN,6.0,49.0,2.0,True,NaN,NaN
16,11-Oct-2023 07:02PM,NaN,http://twitter.com/leand13000/statuses/1712242...,NaN,"I'm in Rockingham County, New Hampshire. Seems...",Twitter,@leand13000,United States,NaN,English,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,False,http://twitter.com/VincentCrypt46/statuses/171...,NaN
21,11-Oct-2023 10:34AM,NaN,http://twitter.com/TownsUsa/statuses/171211443...,NaN,"Tilton Northfield, New Hampshire Belknap Count...",Twitter,@TownsUsa,United States,NaN,English,...,12.0,0.0,0.0,NaN,0.0,12.0,0.0,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5058,19-Oct-2022 10:19AM,NaN,http://twitter.com/nsarwark/statuses/158273833...,NaN,Hillsborough County (NH) had more Superior Cou...,Twitter,@nsarwark,United States,NaN,English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN
5060,18-Oct-2022 09:45PM,NaN,http://twitter.com/UnionLeader/statuses/158254...,NaN,A -year-old former Manchester resident will sp...,Twitter,@UnionLeader,United States,NaN,English,...,7.0,2.0,NaN,NaN,0.0,7.0,2.0,False,NaN,NaN
5061,18-Oct-2022 02:19PM,NaN,http://twitter.com/letshearthetru/statuses/158...,NaN,Santa Clara County appears to be operating in ...,Twitter,@letshearthetru,Unknown,NaN,English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,http://twitter.com/letshearthetru/statuses/158...,NaN
5062,18-Oct-2022 08:46AM,NaN,https://twitter.com/ham_photos_bot/statuses/15...,NaN,"Unidentified Artist, Charity, Public: United S...",Twitter,@ham_photos_bot,United States,NaN,English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN


In [ ]:
#Save cleaned csv to StateName-counties-cleaned.csv
df.to_csv("/content/drive/My Drive/01-Research/Geo-Isa/02-PreprocessedData/NewHampshire-counties-preprocessed.csv")


# Reads all files in a folder; sum the number of rows in all files

In [ ]:
import glob

# 1. list all text files in the directory
rel_filepaths = glob.glob("/content/drive/My Drive/01-Research/Geo-Isa/02-PreprocessedData/*.csv")


In [ ]:
rel_filepaths

['/content/drive/My Drive/01-Research/Geo-Isa/02-PreprocessedData/Alabama-counties-preprocessed.csv',
 '/content/drive/My Drive/01-Research/Geo-Isa/02-PreprocessedData/Alaska-counties-preprocessed.csv',
 '/content/drive/My Drive/01-Research/Geo-Isa/02-PreprocessedData/Arizona-counties-preprocessed.csv',
 '/content/drive/My Drive/01-Research/Geo-Isa/02-PreprocessedData/Arkansas-counties-preprocessed.csv',
 '/content/drive/My Drive/01-Research/Geo-Isa/02-PreprocessedData/California-counties-preprocessed.csv',
 '/content/drive/My Drive/01-Research/Geo-Isa/02-PreprocessedData/Colorado-counties-preprocessed.csv',
 '/content/drive/My Drive/01-Research/Geo-Isa/02-PreprocessedData/Connecticut-counties-preprocessed.csv',
 '/content/drive/My Drive/01-Research/Geo-Isa/02-PreprocessedData/Delaware-counties-preprocessed.csv',
 '/content/drive/My Drive/01-Research/Geo-Isa/02-PreprocessedData/Florida-counties-preprocessed.csv',
 '/content/drive/My Drive/01-Research/Geo-Isa/02-PreprocessedData/Georgia

In [ ]:
# 2. (optional) create a function to read the number of rows in a file
def count_rows(filepath):
  res = 0
  f = open(filepath, 'r')
  res = len(f.readlines())
  f.close()

  return res

# 3. iterate over  files and use the count_row function
counts = [count_rows(filepath) for filepath in rel_filepaths]

print(counts)

[3754, 386, 844, 4521, 4213, 4838, 1448, 1598, 2584, 1432, 1308, 2976, 5570, 4034, 4074, 3177, 8377, 3808, 1812, 3317, 2534, 3262, 4689, 2433, 3939, 2889, 5538, 2161, 4092, 1986, 3003, 4256, 4352, 978, 3089, 5183, 3252, 317, 3533, 1721, 3635, 2470, 7128, 947, 4813, 4822, 5274, 3629, 2511, 530]


In [ ]:
# Python program to find sum of elements in list

total = 0

# Iterate each element in list
# and add them in variable total
for ele in range(0, len(counts)):
    total = total + counts[ele]

# printing total value
print("Sum of all elements in given list: ", total)

Sum of all elements in given list:  163037


# Select Hit Sentence column only

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/My Drive/01-Research/Geo-Isa/02-PreprocessedData/Alabama-counties-preprocessed.csv')


In [ ]:
df = df[['Hit Sentence']]


In [ ]:
df

,Hit Sentence
0,Alabama detention deputy arrested for selling ...
1,"McDonald Chapel, Alabama Jefferson County Popu..."
2,Dave Wenger of Cedar Crest Mennonite Church in...
3,Todays Find: Runaway slave capture notice from...
4,"Live Alabama High School, Girls Volleyball Nor..."
...,...
3748,I worked a few months in limestone county Alab...
3749,Alabama Possible Free College Application Week...
3750,"A FREEZE WARNING is in effect for Benton, Tipp..."
3751,County Arrest Reports /07/22 /15/22


In [ ]:
import glob
import os

In [ ]:
df_list = []

for f in glob.glob("/content/drive/My Drive/01-Research/Geo-Isa/02-PreprocessedData/*.csv"):
        df = pd.read_csv(f)
        filename = (os.path.basename(f))
        df = df[['Hit Sentence']]
        df_list.append(df)

dfData = pd.concat(df_list, ignore_index=True)

In [ ]:
dfData

,Hit Sentence
0,Alabama detention deputy arrested for selling ...
1,"McDonald Chapel, Alabama Jefferson County Popu..."
2,Dave Wenger of Cedar Crest Mennonite Church in...
3,Todays Find: Runaway slave capture notice from...
4,"Live Alabama High School, Girls Volleyball Nor..."
...,...
162982,Hillsborough County (NH) had more Superior Cou...
162983,A -year-old former Manchester resident will sp...
162984,Santa Clara County appears to be operating in ...
162985,"Unidentified Artist, Charity, Public: United S..."
